In [1]:
%pip install --user torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Note: you may need to restart the kernel to use updated packages.


ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


In [2]:
##installs pytorch on a cuda-capable windows machine using pip

%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

%pip install torchsummary

%pip install numpy

%pip install matplotlib

%pip install opencv-python

%pip install scikit-learn

%pip install pandas

%pip install pickle5

Looking in indexes: https://download.pytorch.org/whl/cu118Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pickle5
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pickle5 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\pickle5
      copying pickle5\pickle.py -> build\lib.win-amd64-cpython-311\pickle5
      copying pickle5\pickletools.py -> build\lib.win-amd64-cpython-311\pickle5
      copying pickle5\__init__.py -> build\lib.win-amd64-cpython-311\pickle5
      creating build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\pickletester.py -> build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\test_pickle.py -> build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\__init__.py -> build\lib.win-amd64-cpython-311\pickle5\test
      running build_ext
      building 'pickle5._pickle' exte

In [3]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [4]:
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier


In [5]:
target_dir = os.path.split(os.getcwd())[1]
if(target_dir == "Transfer_Learning" or target_dir == "semi-supervised"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda= torch.cuda.is_available()
print("Cuda is {}".format(cuda))

Current Working Directory: progettoVIPM
Cuda is True


In [6]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.extractNeuralFeatures import extract_features


In [7]:
#Il seguente codice serve per generare un file di annotazione fittizzio, che contenga una mistura tra tutti i dati labelled ed alcuni unlabeled
train_set_labelled = np.loadtxt(datasets.TRAINING_LABELED.value[0], delimiter=",", dtype=np.str_)
train_set_unlabelled = np.loadtxt(datasets.TRAINING_UNLABELED.value[0], delimiter=",", dtype=np.str_)

print("Total labelled examples: {}".format(train_set_labelled.shape[0]))
num_rows = train_set_unlabelled.shape[0]
print("Total unlabelled examples: {}".format(num_rows))
partOfUnlabeledToSelect = 0.10
num_samples = int(num_rows * partOfUnlabeledToSelect)
print("{}% of unlabelled is: {}".format(partOfUnlabeledToSelect*100, num_samples))
np.random.seed(42)
# Get random indices
random_indices = np.random.choice(num_rows, num_samples, replace=False)

# Select the random rows
selected_unlabelled = train_set_unlabelled[random_indices, :]

train_mixed = np.concatenate((train_set_labelled, selected_unlabelled))#prima era X_train_mobi_n, X_unlabled_mobi_n

print("Shape of mixed training set:{}".format(train_mixed.shape))
filename = datasets.TRAINING_MIXED.value[0]
np.savetxt(filename, train_mixed,  delimiter = ",", fmt='%s')
print("Notation file of mixed dataset saved in: {}".format(filename))

Total labelled examples: 5020
Total unlabelled examples: 113455
10.0% of unlabelled is: 11345
Shape of mixed training set:(16365, 2)
Notation file of mixed dataset saved in: train_mixed.csv


In [8]:
# t sta per "tensor", ovvero il vettore sulla gpu, mentre "n" sta per "numpy", ovvero il vettore sulla cpu
chosen_net = networks.MOBILENET
layers_to_remove = 2
X_train_mobi_t, X_train_mobi_n, y_train, X_test_mobi_t, X_test_mobi_n, y_test, fine_tune_layers = extract_features(
                                                                                            cuda=cuda,
                                                                                            transform=None,
                                                                                            train_set=datasets.TRAINING_LABELED,
                                                                                            test_set=datasets.TEST,
                                                                                            network=chosen_net,
                                                                                            layers_to_remove=layers_to_remove)
print("Original classification layers:{}".format(chosen_net.value[1].classifier[:]))
print("Classification layers to fine tune:{}".format(fine_tune_layers[:]))

Found an existing set of features in: ./Storage/neural_features/Train_MobileNetV3_small_minus2_train_set.npy
Loading features from file:
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Train: 5020
Feature vector shape of Train: (5020, 576)
Label vector shape of Train: (5020,)
---------------------------------------------------------------------------------
Found an existing set of features in: ./Storage/neural_features/Test_MobileNetV3_small_minus2_val_set.npy
Loading features from file:
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Test: 11994
Feature vector shape of Test: (11994, 576)
Label vector shape of Test: (11994,)
---------------------------------------------------------------------------------
Original classification layers:Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  

In [10]:
# t sta per "tensor", ovvero il vettore sulla gpu, mentre "n" sta per "numpy", ovvero il vettore sulla cpu
chosen_net = networks.MOBILENET
layers_to_remove = 2
X_mixed_mobi_t, X_mixed_mobi_n, mixed_labels, X_test_mobi_t, X_test_mobi_n, y_test, fine_tune_layers = extract_features(cuda=cuda,
                                                                                        transform=None,
                                                                                        train_set=datasets.TRAINING_MIXED,
                                                                                        test_set=datasets.TEST,
                                                                                        network=chosen_net,
                                                                                        layers_to_remove=layers_to_remove)
print("Original classification layers:{}".format(chosen_net.value[1].classifier[:]))
print("Classification layers to fine tune:{}".format(fine_tune_layers[:]))

Did not find an existing set of features in: ./Storage/neural_features/Train_MobileNetV3_small_minus2_train_mixed.npy
Extracting features:


100%|██████████| 16365/16365 [03:16<00:00, 83.23it/s]


Copying features to cpu:


100%|██████████| 16365/16365 [00:00<00:00, 76076.91it/s]


---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Train: 16365
Feature vector shape of Train: (16365, 576)
Label vector shape of Train: (16365,)
---------------------------------------------------------------------------------
Found an existing set of features in: ./Storage/neural_features/Test_MobileNetV3_small_minus2_val_set.npy
Loading features from file:
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Test: 11994
Feature vector shape of Test: (11994, 576)
Label vector shape of Test: (11994,)
---------------------------------------------------------------------------------
Original classification layers:Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Classification layers to fine tune:S

In [25]:
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def model_building(model, model_name, X_train, X_test, y_train, y_test):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    acc=accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=model.classes_)
    disp.plot()
    plt.savefig("./semi-supervised/model_metrics/ConfM_{}.pdf".format(model_name))
    cm = np.array(cm)
    np.save("./semi-supervised/model_metrics/ConfM_{}.npy".format(model_name), cm)
    with open('./semi-supervised/models/{}.pkl'.format(model_name),'wb') as f:
        pickle.dump(model,f)
    return acc

In [26]:
from sklearn.semi_supervised import LabelSpreading

model  = [LabelSpreading(kernel="knn", max_iter=5, n_jobs=-1, n_neighbors=3), "LabelSpreading"]
model[0].fit(X_mixed_mobi_n, mixed_labels)
print("finito di inferire le labels sugli non annotati")

guessedAndRealLabels = model[0].transduction_


model2 = RandomForestClassifier()
model2.fit(X_mixed_mobi_n, guessedAndRealLabels) 
print("finito di fittare il classificatore")
guessedTestLabels = model2.predict(X_test_mobi_n)
score = accuracy_score(y_test, guessedTestLabels)
cm = confusion_matrix(y_test, guessedTestLabels)
# summarize score
print('Accuracy: %.3f' % (score*100))
np.save("./semi-supervised/model_metrics/Accuracies_{}_minus{}_randomForest_labelSpreading_{}_ofunlabeled.npy".format(chosen_net, layers_to_remove, partOfUnlabeledToSelect), score)
np.save("./semi-supervised/model_metrics/ConfM_{}_minus{}_randomForest_labelSpreading_{}_ofunlabeled.npy".format(chosen_net, layers_to_remove, partOfUnlabeledToSelect), score)
# 5% unlabeled ->6.9 di acc
# 10% unlabeled ->6.9 di acc

finito di inferire le labels sugli non annotati
finito di fittare il classificatore
Accuracy: 6.053
